In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/content/emails.csv")

In [ ]:
import re
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')


nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [ ]:
rows = df.shape[0]
for i in range(rows):
    sentence = df.loc[:, 'text'][i][9:]
    sentence = re.sub(r'[^\w\s]', '', sentence)
    new = ''
    for word in sentence.split(' '):
        if word not in stop:
            new = new + ' ' + str(lemmatizer.lemmatize(word)).lower()
    
    new = re.sub(r"(^|\W)\d+", '', new)

    df['text'][i] = new

In [5]:
#index of all words in vocabulary
word_dict = {}
ind = 0
for i in range(rows):
    sentence = df['text'][i]
    for word in sentence.split(' '):
        if word not in word_dict:
            word_dict[word] = ind
            ind += 1

vocab = len(word_dict.values())

In [6]:
new_list = []
label_list = []
for i in range(rows):
    sentence = df['text'][i]
    a_list = [0]*vocab
    for word in sentence.split(' '):
        word_index = word_dict[word]
        a_list[word_index] += 1
    new_list.append(a_list)
    label_list.append(int(df['spam'][i]))

In [ ]:
new_list = np.array(new_list)
label = np.array(label_list)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_list, label)

In [9]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [10]:
y_pred = clf.predict(X_test)
print(clf.score(X_test, y_test))

0.9776536312849162


In [11]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, f1_score

In [12]:
confusion_matrix(y_test, y_pred)

array([[1093,    2],
       [  30,  307]])

In [13]:
precision_score(y_test, y_pred)

0.9935275080906149

In [14]:
recall_score(y_test, y_pred)

0.9109792284866469

In [15]:
f1_score(y_test, y_pred)

0.9504643962848297